<a href="https://colab.research.google.com/github/HpDiniz/Leitor-de-PDF/blob/main/PDF_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfminer --quiet
!pip install PyPDF2 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 37.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 4.9 MB/s eta 0:00:00


In [2]:
from google.colab import files
file_objects = files.upload() # Total da Carteira 2.998.294,72 3 GAP ABSOLUTO MM SEL

Saving Jun22Santander _ IBPF.pdf to Jun22Santander _ IBPF.pdf


# Gerar consolidado

In [3]:
import io
import re
import os
import uuid
import json
import base64
import PyPDF2
import openpyxl
import requests
import pandas as pd
from datetime import datetime
from flask import Flask, request
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter

In [4]:
def get_response():

    response = []
    for file in file_objects.keys():

        with open(file, "rb") as f:
            pdf_content = f.read()

        base64_pdf_content = base64.b64encode(pdf_content)

        response.append({
            'FileName': file,
            'Institution': "xxx",
            'Value': base64_pdf_content
        })

    return response

def mes_para_numero(data):

    mes, ano = data.split()

    meses = {
        "janeiro": "31/01/" + str(ano),
        "fevereiro": "28/02/" + str(ano),
        "março": "31/03/" + str(ano),
        "abril": "30/04/" + str(ano),
        "maio": "31/05/" + str(ano),
        "junho": "30/06/" + str(ano),
        "julho": "31/07/" + str(ano),
        "agosto": "31/08/" + str(ano),
        "setembro": "30/09/" + str(ano),
        "outubro": "31/10/" + str(ano),
        "novembro": "30/11/" + str(ano),
        "dezembro": "31/12/" + str(ano)
    }
    return meses.get(mes.lower().strip(), None)

# Converte o conteúdo do PDF para texto
def get_pdfminer_text(path):

    retstr = io.StringIO()
    laparams = LAParams()
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)

    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    for page in PDFPage.get_pages(fp, set(), maxpages=0, password="",caching=True, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()

    return text.strip()

def get_pypdf_text(path):

    text, reader = "", PyPDF2.PdfReader(path)
    for page in reader.pages:
        text = text + page.extract_text()

    return text.strip()

def check_ativo_noise(text_pdfminer, remove_from_left, ativo):

    # Remove espaços extras do ativo
    ativo = re.sub('\s+',' ', ativo.strip())

    # Obtém todas as linhas obtidas por uma quebra de linha dupla
    pdfminer_lines = text_pdfminer.split("\n\n")

    # Remove tabs, quebras de linhas e espaços duplos de cada posição
    for i in range(len(pdfminer_lines)):
        pdfminer_lines[i] = re.sub('\s+',' ', pdfminer_lines[i].strip())

    # Cria uma cópia do nome do ativo
    ativo_aux = ativo

    #Loop até quando for possível diminuir o nome do ativo
    while(len(ativo_aux) > 1):

        # Se o nome do ativo não possuir espaços, não há o que fazer
        if ' ' not in ativo_aux:
             return ativo

        # Procura o nome do ativo nas linhas do pdfminer
        if ativo_aux in pdfminer_lines:
            return ativo_aux

        # Remove um caracter do {ativo_aux} e conitnua a busca
        if remove_from_left:
            ativo_aux = ativo_aux[1:]
        else:
            ativo_aux = ativo_aux[:-1]
    
    # Retorna o {ativo}, caso não tenha sido encontrado nada
    return ativo

def obtem_classificacao(ativo):

    # Verifica se é uma ativo no formato de Ticker
    founded = re.search(r'\b[A-Z]{4}(3|4|5|6|11|32|33|34|35)\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"
    
    # Define as palavras chave para cada Classificação
    dict_classificacoes = {
        "Ações": [
            "FIC FIA","FIA","FI Ações", "FI Açoes"
        ],
        "Renda Fixa": [
            "Renda Fixa","RF","CDB","LC","LF","LFSN","LFSC","LCA","LCI","CRA",
            "CRI","Deb","Debenture","Debênture","Tesouro","Pré","Pós","Pre",
            "Pos","IPCA","IPCA+","LTN","NTN","NTN-B","NTNB","DI","REF DI",
            "CP","Credito Privado","Crédito Privado","LP","Longo Prazo"
        ],
        "Renda Fixa (Previdência)": [
            "Previdencia","Previdência","Prev","VGBL","PGBL","VGBL/PGBL"
        ],
        "Multimercado": [
            "FIC FIM","FIM","MM","Multimercado","Multi","COE","Multiestratégia",
            "Multiestratégia"
        ]
    }

    classes = []
    # Percorre cada uma das classificações
    for classificacao in dict_classificacoes:

        # Percorre as palavras-chave de cada uma das classificações
        for key in dict_classificacoes[classificacao]:

            # Verifica se a palavra-chave atual existe no nome do ativo
            founded = re.search(r'\b' + key + r'\b', ativo, flags=(re.IGNORECASE))
            if founded and not classificacao in classes:
                classes.append(classificacao)

    if len(classes) > 0:
        return ' / '.join(classes)

    # Se nenhuma classificação foi encontrada, verifica se é um ETF de investimento estrangeiro
    founded = re.search(r'\b[A-Z]{3}\b', ativo, flags=(re.IGNORECASE))
    if founded:
        return "Ações"

    return "***"

def convert_base64_pdf_to_text(value):

    bytes = base64.b64decode(value, validate=True)

    if bytes[0:4] != b'%PDF':
        raise ValueError('Missing the PDF file signature')

    filename = str(uuid.uuid4()) + '.pdf'

    # Write the PDF contents to a local file
    f = open(filename, 'wb')
    f.write(bytes)
    f.close()

    # Lê o conteúdo do arquivo PDF
    text_pdfminer = get_pdfminer_text(filename)
    text_pypdf = get_pypdf_text(filename)

    os.remove(filename)

    return text_pypdf, text_pdfminer

def json_to_excel(json):

    # Converte o Json para DataFrame
    df = pd.DataFrame(json)
    df.astype({'ATUAL': 'float'}).dtypes

    # Gera um nome de arquivo aleatório
    filename = str(uuid.uuid4()) + '.xlsx'

    # Criar o arquivo Excel
    writer = pd.ExcelWriter(filename, engine='openpyxl')
    df.to_excel(writer, index=False, float_format="%.2f")

    # Ajustar o tamanho das colunas para a largura da maior célula em cada coluna
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    for i, col in enumerate(df.columns):
        column_len = df[col].astype(str).str.len().max()
        column_len = max(column_len, len(col)) + 2
        worksheet.column_dimensions[openpyxl.utils.get_column_letter(i+1)].width = column_len

    # Salvar o arquivo Excel
    writer.close()

    with open(filename, "rb") as f:
        content = f.read()

    os.remove(filename)

    return base64.b64encode(content).decode("utf-8")

def update_file_status(response, file_name, success):

    if 'FileStatus' not in response:
        response['FileStatus'] = []

    response['FileStatus'].append({
        'FileName': file_name,
        'Status': "OK" if success else "ERRO"
    })

def aply_regex_at_position(array, target_index, regex, group_index):

    # Para cada elemento do array, aplica uma regex na posição especificada
    for i in range(len(array)):
        array_list = list(array[i])
        group = re.findall(regex, array_list[target_index], flags=(re.IGNORECASE | re.MULTILINE | re.DOTALL))
        array_list[target_index] = group[group_index]
        array[i] = array_list

def extract_info(result, array, ativo_idx, valor_idx, vencimento_idx, indexador_idx, data_emissao, padrao_extracao, instituicao, noise_config = {}, is_coe = False):

    for item in array:

        # Obtém o valor dos atributos
        ativo = "" if ativo_idx is None else re.sub('\s+',' ', item[ativo_idx].strip())
        classificacao = "Multimercado" if is_coe else obtem_classificacao(ativo)
        valor_atual = "-" if valor_idx is None else re.sub('\s+',' ', item[valor_idx].strip())
        data_vencimento = "-" if vencimento_idx is None else re.sub('\s+',' ', item[vencimento_idx].strip())
        indexador = "-" if indexador_idx is None else re.sub('^\+\s*','', re.sub('\s+',' ', item[indexador_idx].strip()))

        # Garante que o ativo capturado possua letras
        if ativo != '' and not re.search(r'[a-zA-Z]', ativo):
            continue

        # Realiza formatações adicionais
        ativo = "***" if ativo == '' else (check_ativo_noise(noise_config['text'], noise_config['remove_from_left'], ativo) if noise_config != {} else ativo)
        ativo = "COE - " + ativo if is_coe else ativo
        valor_atual = float(valor_atual.replace('.','').replace(',','.')) if re.search(r'\d', valor_atual) else None
        data_vencimento = data_vencimento if re.search(r'\d{2}\/\d{2}\/\d{4}', data_vencimento) else "-"

        result.append({
            'CLASSIFICAÇÃO': classificacao,
            'ATIVO': ativo,
            'EXPOSIÇÃO': "Real",
            'INSTITUIÇÃO': instituicao,
            'ATUAL': valor_atual,
            'DATA': data_emissao,
            'ON/OFF': "ON",
            'DATA DE VENCIMENTO': data_vencimento,
            'INDEXADOR': indexador,
            '[APOIO] ORIGEM EXTRATO': padrao_extracao
        })

In [5]:
def obtem_extrato_cotista_xp(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "XP - Extrato Cotista"

    # Regex para obter a data de emissão
    regex = "(?<=Movimenta..o de \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex, text_pdfminer, flags=(re.IGNORECASE)).group(0)

    # Regex para obter o texto situado entre o termo "POSIÇÃO CONSOLIDADA" e o termo "Emissão"
    regex = "(?<=POSI..O CONSOLIDADA).*?(?=Emissão:)"
    result_1 = re.search(regex, text_pdfminer, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Regex para obter todas as linhas que possuírem pelo menos uma letra e um espaço
    regex = "^.*[a-zA-Z] .*$"
    result_2 = re.finditer(regex, result_1, flags=(re.MULTILINE))

    # Regex para obter o texto situado entre o termo "POSIÇÃOCONSOLIDADA" e o termo "TotalnaInstituição"
    regex = "(?<=POSI..OCONSOLIDADA\n).*?(?=TotalnaInstituição)"
    result_3 = re.search(regex, text_pypdf, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Substitui os espaços por um ponto e vírgula, para simular uma tabela CSV
    result_3 = result_3.replace(" ",";")

    # Corrige termos com espaço
    for result in result_2:
        value = result.group(0)
        key = value.replace(" ","")
        result_3 = result_3.replace(key,value)

    # Converte a tabela CSV para um array de valores
    df_array = pd.read_csv(io.StringIO(result_3), sep=";").to_numpy()
    
    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": True }
    extract_info(result, df_array, 0, 4, None, None, data_emissao, origem_extrato, "XP Investimentos", noise_config)

    return result

def obtem_extrato_consolidado_modal(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "Modal - Extrato Consolidado"

    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": True }

    # Regex para obter a Data de Emissão
    regex = "(?<=Per.odo de refer.ncia \d{2}\/\d{2}\/\d{4} a )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex, text_pdfminer, flags=(re.IGNORECASE)).group(0)

    # Regex para obter todas as rendas fixas
    regex = r'\n[\d\/]*([a-zA-Z].+?)\s*\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*R\$\s*[\d,.]+[.,]\d{2}\s*[\d.]+(\s*)R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}\s*[\d.,]+'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 3, 1, None, data_emissao, origem_extrato, "Modal", noise_config)   

    # Regex para obter todas as ações
    regex = r'(BRUTO|\,\d\d)\s*([a-zA-Z].+?\-(.|\n)*?[A-Z]{4}(3|4|5|6|11|32|33|34|35))\s*[\d.]+\s*R\$\s*[\d.]+\,\d\d\s*R\$\s*([\d.]+\,\d\d)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 1, 4, None, None, data_emissao, origem_extrato, "Modal", noise_config)
    
    # Regex para obter todas os CRAs
    regex = r'(L.QUIDO|\,\d\d)\s*([a-zA-Z].+?\-(.|\n)*?)\d{2}\/\d{2}\/\d{4}\s*[\d.]+\s*R\$\s*([\d.]+\,\d\d)\s*R\$\s*[\d.]+\,\d\d\s*R\$\s*[\d.]+\,\d\d'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 1, 3, None, None, data_emissao, origem_extrato, "Modal", noise_config)

    return result

def obtem_posicao_performance_xp(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "XP - Posição e Performance"

    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": True }

    # Regex para obter a Data de Emissão
    regex = "(?<=Data da consulta: )\d\d\/\d\d\/\d{4}"
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0)
    
    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4}|\-)\s*(\d{2}\/\d{2}\/\d{4})([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s]*)([\d.]+\,\d\d)[\d.]+\,\d\d'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 5, 3, 4, data_emissao, origem_extrato, "XP Investimentos", noise_config)

    # Regex para obter todas as linhas que possuírem ativos de COE
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}[\s\d.]+\,\d\d([\d.]+\,\d\d)[\d.]+\,\d\d'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 2, None, None, data_emissao, origem_extrato, "XP Investimentos", noise_config)

    # Regex para obter todas as linhas que possuírem ativos de COE
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*[\d.]+\,\d\d[\d.]+\,\d\d([\d.]+\,\d\d)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))   
    extract_info(result, tuple_array, 0, 3, 2, None, data_emissao, origem_extrato, "XP Investimentos", noise_config, is_coe = True)

    # Regex para obter todas as linhas que possuírem ativos de Previdência
    regex = r'(((?!\d{2}\/\d{2}\/\d{4}).)*)\d{2}\/\d{2}\/\d{4}\s*[\d.]+\,\d{2}\s*[\d.]+\s*([\d.]+\,\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))  
    extract_info(result, tuple_array, 0, 2, None, None, data_emissao, origem_extrato, "XP Investimentos", noise_config)

    # Regex para obter todas as linhas que possuírem ativos de ??
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))[\d\s.]*\,\d\d\s*[\d.]+\,\d\d\s*[\d\-\+]+\,\d\d\s*([\d.]+\,\d\d)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))   
    extract_info(result, tuple_array, 0, 2, None, None, data_emissao, origem_extrato, "XP Investimentos", noise_config)    

    return result

def obtem_posicao_consolidada_xp(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "XP - Posição Consolidada"

    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": True }

    # Regex para obter a Data de Emissão
    regex = "(?<=Data da Consulta: )\d{2}\/\d{2}\/\d{4}"
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0)

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'\n[\d\/]*([A-Z].+?\s*-\s*[A-Z]{3}\/\d{4})\s*\d{2}\/\d{2}\/\d{4}\s*\d{2}\/\d{2}\/\d{4}\s*(\d{2}\/\d{2}\/\d{4})\s*([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s]*)\d+\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 3, 1, 2, data_emissao, origem_extrato, "XP Investimentos", noise_config)   

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa sem carência
    regex = r'\n[\d\/]*([A-Z].+?\s*-\s*[A-Z]{3}\/\d{4})\s*\d{2}\/\d{2}\/\d{4}\s*\-\s*(\d{2}\/\d{2}\/\d{4})\s*([A-Z\-\+\s]*\s*[\d,.]+%[A-Z\-\s]+)\d+\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*([\d,.]+[.,]\d{2})\s*R\$\s*[\d,.]+[.,]\d{2}'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 3, 1, 2, data_emissao, origem_extrato, "XP Investimentos", noise_config)  

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa Pós-Fixados
    regex = r'\n[\d\/]*([A-Z]((?!\d{2}\/\d{2}\/\d{4}).)+)\s*\d{2}\/\d{2}\/(\d{4})\s*[\d,.]+\s*[\d,.]+\s*R\$\s*[\d,.]+(\s*)R\$\s*([\d,.]+)\s*R\$\s*[\d,.]+'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 4, None, None, data_emissao, origem_extrato, "XP Investimentos", noise_config)  

    # Regex para obter todas as linhas que possuírem Fundos Imobiliários
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}(\s*)R\$\s*([\d,.]+[.,]\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 3, None, None, data_emissao, origem_extrato, "XP Investimentos", noise_config)

    # Regex para obter todas as linhas que possuírem COEs
    regex = r'\n[\d\/]*([A-Z]((?!\d{2}\/\d{2}\/\d{4}).)+)\s*-\s*[\w\s,.]*\s*-\s*\s*\d{2}\.\d{2}\.\d{4}\s*[\w\s,.]*\d{2}\/\d{2}\/\d{4}\s+(\d{2}\/\d{2}\/\d{4})\s+\d+\s*R\$\s*[\d,.]+[.,]\d{2}\s*R\$\s*[\d,.]+[.,]\d{2}(\s*)R\$\s*([\d,.]+[.,]\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 4, 2, None, data_emissao, origem_extrato, "XP Investimentos", noise_config, is_coe = True)

    # Regex para obter todas as linhas que possuírem Ações
    regex = r'([A-Z]{4}(3|4|5|6|11|32|33|34|35))\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+\d+\s+R\$\s*[\d,.]+[.,]\d{2}(\s*)R\$\s*([\d,.]+[.,]\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 3, None, None, data_emissao, origem_extrato, "XP Investimentos", noise_config)

    return result

def obtem_posicao_consolidada_genial(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "Genial - Posição Consolidada"

    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": True }
    
    # Regex para obter a Data de Emissão
    regex = r'(?<=Emissão:\xa0)\d{2}\/\d{2}\/\d{4}'
    data_emissao = re.search(regex, text_pdfminer, flags=(re.IGNORECASE)).group(0)

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'(((?!MULTIMERCADO|RENDA FIXA|PREVIDÊNCIA|AÇÕES).)*).+?\s*\d{2}\/\d{2}\/\d{4}\s*R\$([\d.]+\,\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 2, None, None, data_emissao, origem_extrato, "Genial", noise_config)

    # Regex para obter todas as linhas que possuírem ativos de Renda Fixa
    regex = r'^([A-Z].*?)\s*[\d.]+\,\d{2}\s*[\d.]+\,\d{2}\s*[\d.]+\,\d{2}\s*(\d{2}\/\d{2}\/\d{4})\s*R\$\s*([\d.]+\,\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 2, 1, None, data_emissao, origem_extrato, "Genial", noise_config)

    return result

def obtem_carteira_investimentos_itau(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "Itau - Carteira de Investimentos" 

    # Cria o array resultante
    result = []

    # Regex para obter a Data de Emissão
    regex = r'(janeiro|fevereiro|março|abril|maio|junho|julho|agosto|setembro|outubro|novembro|dezembro)\s\d{4}'
    data_emissao = mes_para_numero(re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0))
    
    # Regex para obter a Data de Emissão
    regex = r'((.*\n){5})([\d.]+\,\d{2})\s*([\- \d\%]+|\d{2}\/\d{2}\/\d{2})\s*([\- \d\%]+|\d{2}\/\d{2}\/\d{2})\s*([\d\%A-Z\+\-\,\.]+\s*[\d\%A-Z\+\-\,\.]*?)\s*\d{0,3}\%\s*(Alto|Baixo|M.dio)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    aply_regex_at_position(tuple_array, 0, r'.*\n[^a-zA-Z]+$(.*)', 0)
    extract_info(result, tuple_array, 0, 2, 4, 5, data_emissao, origem_extrato, "Itaú")

    return result

def obtem_carteira_detalhada_itau(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "Itau - Carteira Detalhada"

    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": True }

    # Regex para obter a Data de Emissão
    regex = r'(?<=Atualizado em )\d{2}\/\d{2}\/\d{4}\s*(?=Hist.rico de Rentabilidade)'
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Regex para obter o bloco onde estão os ativos
    regex = r'(?<=Carteira detalhada).*(?=Histórico de Rentabilidade)'
    ativos_text = re.search(regex, text_pypdf, flags=(re.IGNORECASE | re.DOTALL)).group(0)

    # Regex para obter os ativos
    regex = r'(^.*?)\s*([\d.]+\,\d{2})\s*[\d]+\,\d{2}'
    tuple_array = re.findall(regex, ativos_text, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 1, None, None, data_emissao, origem_extrato, "Itaú", noise_config)
    
    return result

def obtem_posicao_consolidada_itau(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "Itau - Posição Consolidada"

    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": True }

    # Regex para obter a Data de Emissão
    regex = r'(?<=emitido em )\d{2}\/\d{2}\/\d{4}'
    data_emissao = re.search(regex, text_pdfminer, flags=(re.IGNORECASE)).group(0)

    # Regex para obter os ativos
    regex = r'((.*\n){3,8}.*)\s*(R\$\s*[\d.]+\,\d{2}\s*[\d,.]+\%){5}([\d.]+\,\d{2})'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 3, None, None, data_emissao, origem_extrato, "Itaú", noise_config)

    return result

def obtem_extrato_movimentacao_bradesco(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "Bradesco - Extrato de Movimentação"

    # Cria o array resultante
    result = []

    # Regex para obter a Data de Emissão
    regex = r'(?<=Até\:)\d{2}\/\d{2}\/\d{4}'
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0)

    # Regex para obter o nome do ativo
    regex = r'(Produto|Nome do Fundo): (.*)\s*(.|\n)*Total saldo atual\s*([\d.,]+)\s*([\d.,]+)\s*([\d.,]+)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE))

    if re.search(r'Vlr\s*Princ\.\s*\(R\$\)\s*Vlr\.\s*Bruto\s*\(R\$\)', text_pypdf, flags=(re.IGNORECASE)):
        extract_info(result, tuple_array, 1, 4, None, None, data_emissao, origem_extrato, "Bradesco")
    elif re.search(r'Valor\s*da\s*Cota\s*Vlr\.\s*Bruto\s*\(R\$\)', text_pypdf, flags=(re.IGNORECASE)):
        extract_info(result, tuple_array, 1, 5, None, None, data_emissao, origem_extrato, "Bradesco")

    return result

def obtem_relatorio_carteira_santander(text_pypdf, text_pdfminer):

    # Define o nome do padrão de extração
    origem_extrato = "Santander - Relatório Carteira"

    # Cria o array resultante
    result = []

    # Define a configuração para remoção de ruídos do nome do ativo
    noise_config = {"text": text_pdfminer, "remove_from_left": False}

    # Regex para obter a Data de Emissão
    regex = r'(?<=Data da P osição: )\d{2}\/\d{2}\/\d{4}'
    data_emissao = re.search(regex, text_pypdf, flags=(re.IGNORECASE)).group(0)

    # Regex para obter os ativos de Renda Fixa
    regex = r'\d{2}\/\d{2}\/\d{2}\s*(.*?)\(*(\d+\,\d+\s*(.|\n)*?)\)*\d{2}\/\d{2}\/\d{2}\s*(\d{2}\/\d{2}\/\d{2})\s*([\-\d\,\.]+\s*){4}([\-\d\,\.]+)'
    tuple_array = re.findall(regex, text_pypdf, flags=(re.IGNORECASE | re.MULTILINE))
    extract_info(result, tuple_array, 0, 5, 2, 1, data_emissao, origem_extrato, "Santander", noise_config)    

    return result
    
def execute_extraction(text_pypdf, text_pdfminer):

    # Verifica se há caracteres mínimos
    if(len(text_pdfminer) > 99):

        # Armazena os primeiros caracteres do pdf
        text_header = re.sub('\s+','', text_pdfminer[0:100].lower())

        # Mapeia todos os métodos de extração de acordo com uma expressão regular
        extracoes_map = [
            (r'extratodecotista', obtem_extrato_cotista_xp),
            (r'posi..o&performance', obtem_posicao_performance_xp),
            (r'posi..oconsolidadadatada', obtem_posicao_consolidada_xp),
            (r'contaxp.*?códigoassessor', obtem_posicao_consolidada_xp),
            (r'extratoconsolidadoinvestimentos', obtem_extrato_consolidado_modal),
            (r'dispon.velparainvestirr\$[\d.,]+aliquidar', obtem_posicao_consolidada_genial),
            (r'carteiradeinvestimentos', obtem_carteira_investimentos_itau),
            (r'nomeclienteresumosaldoparaaplica..o', obtem_carteira_detalhada_itau),
            (r'posi..oconsolidadatipodeinvest', obtem_posicao_consolidada_itau),
            (r'extratodemovimentaçãonome', obtem_extrato_movimentacao_bradesco),
            (r'santander.*?relat.riodecarteira', obtem_relatorio_carteira_santander)
        ]

        # Faça a extração de acordo com o método da expressão regular que houver o match
        for extracao in extracoes_map:
            if re.search(extracao[0], text_header):
                return extracao[1](text_pypdf, text_pdfminer)

    return []


In [6]:
# @Flask(__name__).route('/obtem-ativos', methods=['POST'])
def obtem_ativos():

    # Obtém a requisição 
    request_data = get_response() #json.loads(request.data)

    # Cria as variáveis resultantes
    result, response = [], {"FileStatus": [], "Value": ""}

    # Percorre cada um dos arquivos do array
    for res in request_data:
        
        # Obtém o texto presente no pdf 
        file_name = res['FileName']
        text_pypdf, text_pdfminer = convert_base64_pdf_to_text(res['Value'])

        # Obtém o tamanho do array antes da inserção dos novos ativos
        len_before = len(result)

        # Executa a extração do PDF
        # try:
        result = result + execute_extraction(text_pypdf, text_pdfminer)
        # except Exception as e:
        #     print(e)

        # Atualiza o status de extração do arquivo atual
        update_file_status(response, file_name, (len(result) > len_before))

    # Caso o tamanho do DataFrame seja maior que zero, retorna o base64 do consolidado
    if (len(result) > 0):
        response["Value"] = json_to_excel(result)

    return response

In [7]:
response = obtem_ativos()

In [8]:
response

{'FileStatus': [{'FileName': 'Jun22Santander _ IBPF.pdf', 'Status': 'OK'}],
 'Value': 'UEsDBBQAAAAIACWZTlYHQU1igQAAALEAAAAQAAAAZG9jUHJvcHMvYXBwLnhtbE2OPQsCMRBE/8pxvbdBwUJiQNBSsLIPexsvkGRDskJ+vjnBj24ebxhG3wpnKuKpDi2GVI/jIpIPABUXirZOXaduHJdopWN5ADvnkc6Mz0hJYKvUHqgJpZnmTf4Ojkafcg4erXhO5uqxcGUnw6UhBQ3/cm3eqdQ17yb1lh/W8DtpXlBLAwQUAAAACAAlmU5W7T4uXe8AAAArAgAAEQAAAGRvY1Byb3BzL2NvcmUueG1szZJRS8MwEMe/iuS9vaaVoaHLi2NPCoIDxbeQ3LZgk4bkpN23N61bh+gH8DF3//zud3CtDkL3EZ9jHzCSxXQzus4nocOaHYmCAEj6iE6lMid8bu776BTlZzxAUPpDHRDqqlqBQ1JGkYIJWISFyGRrtNARFfXxjDd6wYfP2M0wowE7dOgpAS85MDlNDKexa+EKmGCE0aXvApqFOFf/xM4dYOfkmOySGoahHJo5l3fg8Pb0+DKvW1ifSHmN+Veygk4B1+wy+bV52Oy2TNZV3RRVXfDbHb8X1Z3gq/fJ9YffVdj1xu7tPza+CMoWft2F/AJQSwMEFAAAAAgAJZlOVplcnCMQBgAAnCcAABMAAAB4bC90aGVtZS90aGVtZTEueG1s7Vpbc9o4FH7vr9B4Z/ZtC8Y2gba0E3Npdtu0mYTtTh+FEViNbHlkkYR/v0c2EMuWDe2STbqbPAQs6fvORUfn6Dh58+4uYuiGiJTyeGDZL9vWu7cv3uBXMiQRQTAZp6/wwAqlTF61WmkAwzh9yRMSw9yCiwhLeBTL1lzgWxovI9bqtNvdVoRpbKEYR2RgfV4saEDQVFFab18gtOUfM/gVy1SNZaMBE1dBJrmItPL5b

In [9]:
if response['Value'] != '':

    f = open("Ativos.xlsx", 'wb')
    f.write(base64.b64decode(response['Value'], validate=True))
    f.close()
    
    files.download("Ativos.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [76]:
get_xx("ASFIASF AS ASI Deb CRI LCI LFSC FI Açoes Multiestratégia BCFF3 TORD11")

FIM
Previdência
Prev
VGBL/PGBL
MFII11
TGAR11
FIC FIM
COE
Previdencia
VGBL
PGBL
TORD11


'Renda Fixa - Líquido'

In [15]:
df = pd.DataFrame(new_tipos)

In [17]:
df.sort_values(by='Regra Prioritária', ascending=False)

,Classe do Ativo,Tipo do Ativo,Termo Buscado,Regra Prioritária
0,Multimercado,Fundos Multimercado,FIM,True
36,Multimercado,Certificado de Operações Estruturadas,COE,True
38,Renda Fixa - Previdência,Fundos de Previdência,VGBL,True
39,Renda Fixa - Previdência,Fundos de Previdência,PGBL,True
32,Multimercado,Fundos Multimercado,FIC FIM,True
...,...,...,...,...
29,Ações,Brazilian Depositary Receipts (BDR),"""xxxx33""",False
30,Ações,Brazilian Depositary Receipts (BDR),"""xxxx34""",False
31,Ações,Brazilian Depositary Receipts (BDR),"""xxxx35""",False
1,Multimercado,Fundos Multimercado,Multi,False


'[A-Z][A-Z][A-Z][A-Z]\\d\\d'